In [1]:
import torch
import torch.nn as nn

# from drdmannturb.calibration import CalibrationProblem
# from drdmannturb.data_generator import OnePointSpectraDataGenerator
from drdmannturb.parameters import (
    LossParameters,
    NNParameters,
    PhysicalParameters,
    ProblemParameters,
)
from drdmannturb.spectra_fitting import CalibrationProblem, OnePointSpectraDataGenerator

device = "cuda" if torch.cuda.is_available() else "cpu"

# v2: torch.set_default_device('cuda:0')
if torch.cuda.is_available():
    torch.set_default_tensor_type("torch.cuda.FloatTensor")

L = 0.59


In [2]:


Gamma = 3.9
sigma = 3.4

domain = torch.logspace(-1, 2, 20)

pb = CalibrationProblem(
    nn_params=NNParameters(
        nlayers=2,
        hidden_layer_sizes=[10, 10],
        activations=[nn.GELU(), nn.GELU()],
    ),
    prob_params=ProblemParameters(nepochs=5),
    loss_params=LossParameters(alpha_pen2=1.0, alpha_pen1=1.0e-5, beta_reg=2e-4),
    phys_params=PhysicalParameters(L=L, Gamma=Gamma, sigma=sigma, domain=domain),
    device=device,
)

k1_data_pts = domain
DataPoints = [(k1, 1) for k1 in k1_data_pts]

Data = OnePointSpectraDataGenerator(data_points=DataPoints).Data

In [3]:
pb.eval(k1_data_pts)
optimal_parameters = pb.calibrate(data=Data)

Initial loss: 0.1271737205129384


100%|██████████| 5/5 [00:13<00:00,  2.60s/it]

Spectral fitting concluded with final loss: 0.008182450856447906
